# **Bước 0: Import thư viện**

In [41]:
import keras
import os
import glob
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import shutil
from tensorflow.keras import models, layers,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Bước 1: Đọc ảnh và file JSON**

In [42]:
# Đường dẫn thư mục dữ liệu gốc
IMAGE_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
JSON_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"

# Thư mục lưu kết quả sau khi xử lý
OUTPUT_IMAGE_DIR = "/kaggle/working/processed_images"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

if not os.path.exists(OUTPUT_IMAGE_DIR):
    os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_IMAGE_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_IMAGE_DIR}")

if not os.path.exists(OUTPUT_JSON_DIR):
    os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_JSON_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_JSON_DIR}")

✅ Thư mục đã tồn tại: /kaggle/working/processed_images
✅ Thư mục đã tồn tại: /kaggle/working/processed_annotations


# Bước 2: Tiền xử lý dữ liệu

Resize ảnh về kích thước cố định nhưng không méo hình.  
Padding ảnh để giữ nguyên tỷ lệ.  
Cập nhật tọa độ vùng ung thư trong JSON để không bị sai lệch.  

In [43]:
# Kích thước ảnh mong muốn
TARGET_SIZE = (224, 294)  # (width, height)
def resize_and_pad_image(img_path, json_path=None):
    """ Resize ảnh về kích thước chuẩn và cập nhật tọa độ vùng ung thư trong JSON (nếu có). """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape[:2]

    # Tính tỷ lệ resize để không làm méo ảnh
    scale = min(TARGET_SIZE[0] / w, TARGET_SIZE[1] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    
    # Resize ảnh
    resized_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Tạo ảnh nền trắng (uint8)
    final_img = np.ones((TARGET_SIZE[1], TARGET_SIZE[0]), dtype=np.uint8) * 255
    x_offset = (TARGET_SIZE[0] - new_w) // 2
    y_offset = (TARGET_SIZE[1] - new_h) // 2

    # Đưa ảnh vào trung tâm
    final_img[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_img

    # Cập nhật JSON nếu có
    updated_data = None
    if json_path and os.path.exists(json_path):
        with open(json_path, "r") as f:
            data = json.load(f)

        # Cập nhật tọa độ vùng tổn thương
        for shape in data["shapes"]:
            updated_points = [
                [round(point[0] * scale + x_offset), round(point[1] * scale + y_offset)]
                for point in shape["points"]
            ]
            shape["points"] = updated_points

        updated_data = data

    return final_img, updated_data

# Xử lý toàn bộ ảnh
image_paths = glob.glob(IMAGE_DIR + "/*.png") + glob.glob(IMAGE_DIR + "/*.jpg") + glob.glob(IMAGE_DIR + "/*.jpeg")

for img_path in image_paths:
    filename = os.path.basename(img_path).split('.')[0]  # Lấy tên file không có đuôi
    json_path = os.path.join(JSON_DIR, filename + ".json")  # Tìm file JSON tương ứng

    # Resize ảnh & cập nhật JSON
    processed_img, updated_annotations = resize_and_pad_image(img_path, json_path if os.path.exists(json_path) else None)

    # Lưu ảnh đã resize
    output_img_path = os.path.join(OUTPUT_IMAGE_DIR, filename + ".jpg")
    cv2.imwrite(output_img_path, processed_img)

    # Lưu JSON nếu có
    if updated_annotations:
        output_json_path = os.path.join(OUTPUT_JSON_DIR, filename + ".json")
        with open(output_json_path, "w") as f:
            json.dump(updated_annotations, f, indent=4)

print("✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại:", OUTPUT_IMAGE_DIR, "và", OUTPUT_JSON_DIR)


✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại: /kaggle/working/processed_images và /kaggle/working/processed_annotations


# **Bước 3: Phân chia bộ test và train**

Phân chia ảnh thành 80% train và 20% test

In [44]:
OUTPUT_DIR = "/kaggle/working/"
# Tạo thư mục train, test, val
for folder in ["train", "test", "val"]:
    os.makedirs(os.path.join(OUTPUT_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh
image_paths = glob.glob(os.path.join(OUTPUT_IMAGE_DIR, "*.jpg")) + \
              glob.glob(os.path.join(OUTPUT_IMAGE_DIR, "*.jpeg"))

# Shuffle dữ liệu để chia ngẫu nhiên
random.seed(42)
random.shuffle(image_paths)

# Chia theo tỷ lệ 70% train - 20% test - 10% validation
train_idx = int(0.7 * len(image_paths))
test_idx = int(0.9 * len(image_paths))  # Sau train, còn 30%, chia tiếp thành 20% test + 10% val

train_images = image_paths[:train_idx]
test_images = image_paths[train_idx:test_idx]
val_images = image_paths[test_idx:]

# Di chuyển ảnh vào thư mục tương ứng
for img_path in train_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "train", os.path.basename(img_path)))

for img_path in test_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "test", os.path.basename(img_path)))

for img_path in val_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "val", os.path.basename(img_path)))

print("✅ Chia tập dữ liệu hoàn tất!")

✅ Chia tập dữ liệu hoàn tất!


In [45]:
import os

test_path = "/kaggle/working/test" 
test_files = len(os.listdir(test_path))
print(f"📂 Số lượng file trong {test_path}: {test_files}")

benign_path = "/kaggle/working/train" 
benign_files = len(os.listdir(benign_path)) 
print(f"📂 Số lượng file trong {benign_path}: {benign_files}")

malignant_path = "/kaggle/working/val"
malignant_files = len(os.listdir(malignant_path)) 
print(f"📂 Số lượng file trong {malignant_path}: {malignant_files}")

📂 Số lượng file trong /kaggle/working/test: 751
📂 Số lượng file trong /kaggle/working/train: 2624
📂 Số lượng file trong /kaggle/working/val: 375


In [46]:
# Thư mục train gốc
TRAIN_DIR = "/kaggle/working/train"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

# Tạo thư mục benign/malignant trong train
for folder in ["benign", "malignant"]:
    os.makedirs(os.path.join(TRAIN_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh trong train
train_images = glob.glob(os.path.join(TRAIN_DIR, "*.jpg")) + \
               glob.glob(os.path.join(TRAIN_DIR, "*.jpeg"))

# Phân loại ảnh
for img_path in train_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    if os.path.exists(json_path):
        dest_folder = "malignant"  # Có JSON -> malignant
    else:
        dest_folder = "benign"  # Không có JSON -> benign

    shutil.move(img_path, os.path.join(TRAIN_DIR, dest_folder, filename))

print("✅ Phân loại ảnh trong train hoàn tất!")

✅ Phân loại ảnh trong train hoàn tất!


In [47]:
# Thư mục train gốc
TEST_DIR = "/kaggle/working/test"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

# Tạo thư mục benign/malignant trong train
for folder in ["benign", "malignant"]:
    os.makedirs(os.path.join(TEST_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh trong train
train_images = glob.glob(os.path.join(TEST_DIR, "*.jpg")) + \
               glob.glob(os.path.join(TEST_DIR, "*.jpeg"))

# Phân loại ảnh
for img_path in train_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    if os.path.exists(json_path):
        dest_folder = "malignant"  # Có JSON -> malignant
    else:
        dest_folder = "benign"  # Không có JSON -> benign

    shutil.move(img_path, os.path.join(TEST_DIR, dest_folder, filename))

print("✅ Phân loại ảnh trong train hoàn tất!")

✅ Phân loại ảnh trong train hoàn tất!


In [48]:
for folder in ["benign", "malignant"]:
    count = len(glob.glob(os.path.join(TRAIN_DIR, folder, "*.jpg")))
    print(f"📂 Số lượng ảnh trong train/{folder}: {count}")

for folder in ["benign", "malignant"]:
    count = len(glob.glob(os.path.join(TEST_DIR, folder, "*.jpg")))
    print(f"📂 Số lượng ảnh trong TEST/{folder}: {count}")

📂 Số lượng ảnh trong train/benign: 1308
📂 Số lượng ảnh trong train/malignant: 1314
📂 Số lượng ảnh trong TEST/benign: 378
📂 Số lượng ảnh trong TEST/malignant: 371


In [49]:
!zip -r /kaggle/working/btxrd_v1.zip /kaggle/working/

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/processed_data.zip (stored 0%)
updating: kaggle/working/btxrd_v2.zip (stored 0%)
updating: kaggle/working/val/ (stored 0%)
updating: kaggle/working/val/IMG000248.jpg (deflated 5%)
updating: kaggle/working/val/IMG003172.jpg (deflated 1%)
updating: kaggle/working/val/IMG000254.jpg (deflated 8%)
updating: kaggle/working/val/IMG000874.jpg (deflated 1%)
updating: kaggle/working/val/IMG001805.jpg (deflated 2%)
updating: kaggle/working/val/IMG002209.jpg (deflated 3%)
updating: kaggle/working/val/IMG003114.jpg (deflated 6%)
updating: kaggle/working/val/IMG002370.jpg (deflated 4%)
updating: kaggle/working/val/IMG000624.jpg (deflated 0%)
updating: kaggle/working/val/IMG003468.jpg (deflated 4%)
updating: kaggle/working/val/IMG001305.jpg (deflated 1%)
updating: kaggle/working/val/IMG002898.jpg (deflated 5%)
updating: kaggle/working/val/IMG002312.jpg (deflated 10%)
updating: kaggle/working/val/IMG002779.jpg (deflated 2%)
updating: kaggl

In [60]:
cp /kaggle/working/btxrd_v1.zip /kaggle/input/



cp: cannot create regular file '/kaggle/input/btxrd_v1.zip': Read-only file system


# **Bước 4: Xây dựng hệ thống CNN**

In [50]:
# Định nghĩa ImageDataGenerator cho train + validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 🔥 Chia 20% từ train làm validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Chỉ cần chuẩn hóa cho test

# Load ảnh từ thư mục train (chia train/val tự động)
train_generator = train_datagen.flow_from_directory(
    "/kaggle/working/train",  
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    subset="training"  # 🔥 Chỉ lấy phần training (80%)
)

val_generator = train_datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    subset="validation"  # 🔥 Chỉ lấy phần validation (20%)
)

# Load ảnh từ thư mục test
test_generator = test_datagen.flow_from_directory(
    "/kaggle/working/test",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    shuffle=False  # Không shuffle để đảm bảo đánh giá đúng
)

print("✅ Load dữ liệu hoàn tất!")

Found 2099 images belonging to 2 classes.
Found 523 images belonging to 2 classes.
Found 749 images belonging to 2 classes.
✅ Load dữ liệu hoàn tất!


In [51]:
model = models.Sequential([
    Input(shape=(224, 294, 3)),  
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Sửa lại số lớp đầu ra
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 224, 294, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 224, 294, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 112, 147, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 112, 147, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 112, 147, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 112, 147, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 56, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 56, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 56, 73, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 56, 73, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 28, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 28, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 28, 36, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 28, 36, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 14, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 14, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 64512)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      33,030,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,203,938 (130.48 MB)

 Trainable params: 34,203,938 (130.48 MB)

 Non-trainable params: 0 (0.00 B)

# **Huấn luyện mô hình**

In [52]:
# model.fit(train_generator,
#           epochs=20,
#           validation_data=val_generator)

In [53]:
# model.save('/kaggle/working/model.h5')  # Lưu mô hình
# print("Mô hình đã được lưu thành công!")

In [54]:
# from IPython.display import FileLink

# FileLink("/kaggle/working/model.h5")  # Hiển thị link tải file